# Intro

Many of the tags in the corpus are more or less equivalent (e.g. "great cinemaphotography", "beautifully shot").

The goal here is to discover which tags are most correlated and only use one of each pair to increase information in the tag set.

In [1]:
%load_ext google.cloud.bigquery

import pandas as pd

In [2]:
! bq ls movielens

/Users/patrickdinneen/google-cloud-sdk/platform/bq/bq.py:41: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  import imp
         tableId          Type    Labels   Time Partitioning   Clustered Fields  
 ----------------------- ------- -------- ------------------- ------------------ 
  movie_tags              TABLE                                                  
  movie_tags_unit_score   TABLE                                                  
  movies                  TABLE                                                  
  tags                    TABLE                                                  


In [3]:
! bq show movielens.movie_tags

/Users/patrickdinneen/google-cloud-sdk/platform/bq/bq.py:41: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  import imp
Table pjd-hosting:movielens.movie_tags

   Last modified          Schema          Total Rows   Total Bytes   Expiration   Time Partitioning   Clustered Fields   Labels  
 ----------------- --------------------- ------------ ------------- ------------ ------------------- ------------------ -------- 
  24 Aug 08:41:35   |- tag: string        10551655     295027780                                                                 
                    |- item_id: integer                                                                                          
                    |- score: float                                                                                              



In [8]:
%%bigquery tag_scores

SELECT 
    * 
FROM movielens.movie_tags 
WHERE item_id IN (
    SELECT
        item_id
    FROM movielens.movies
    ORDER BY RAND()
    LIMIT 10000
);

/Users/patrickdinneen/GD_Dev/ext/movie-explorer/.venv/lib/python3.7/site-packages/google/cloud/bigquery/job/query.py:1683: UserWarning: A progress bar was requested, but there was an error loading the tqdm library. Please install tqdm to use the progress bar functionality.
  query_result = wait_for_query(self, progress_bar_type, max_results=max_results)
/Users/patrickdinneen/GD_Dev/ext/movie-explorer/.venv/lib/python3.7/site-packages/google/cloud/bigquery/table.py:1942: UserWarning: A progress bar was requested, but there was an error loading the tqdm library. Please install tqdm to use the progress bar functionality.
  create_bqstorage_client=create_bqstorage_client,


In [11]:
tag_scores_wide = tag_scores.pivot(index="item_id", columns="tag", values="score")

In [12]:
correlation_matrix = tag_scores_wide.corr()

In [13]:
print(correlation_matrix)

tag                007  007 (series)  18th century     1920s     1930s  \
tag                                                                      
007           1.000000      0.984736      0.066076 -0.023022 -0.029069   
007 (series)  0.984736      1.000000      0.055972 -0.026003 -0.026403   
18th century  0.066076      0.055972      1.000000  0.338456  0.209524   
1920s        -0.023022     -0.026003      0.338456  1.000000  0.428615   
1930s        -0.029069     -0.026403      0.209524  0.428615  1.000000   
...                ...           ...           ...       ...       ...   
writers      -0.050872     -0.048752     -0.135796 -0.102410 -0.010194   
wuxia        -0.002019     -0.005360      0.139079  0.023841  0.001794   
wwii         -0.019922     -0.016344      0.056484  0.063923  0.222043   
zombie       -0.030199     -0.028460     -0.104038 -0.076157 -0.084940   
zombies      -0.012740     -0.012012     -0.050165 -0.017035 -0.042976   

tag              1950s     1960s     

In [14]:
type(correlation_matrix)

pandas.core.frame.DataFrame

In [16]:
stacked = correlation_matrix.rename_axis(None).rename_axis(None, axis=1)

In [17]:
stacked = stacked.stack().reset_index()

In [20]:
stacked.columns = ["lhs_tag", "rhs_tag", "correlation"]

In [29]:
stacked[(stacked.lhs_tag != stacked.rhs_tag)][stacked.correlation > 0.8].to_csv("tag_correlations.csv", index=False)

/Users/patrickdinneen/GD_Dev/ext/movie-explorer/.venv/lib/python3.7/site-packages/ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.
